In [1]:
import squidpy as sq
import scanpy as sc
import pandas as pd
import numpy as np
from scipy.sparse import csr_matrix, isspmatrix_csr

/home/dbdimitrov/anaconda3/lib/python3.8/site-packages/paramiko/transport.py:219: CryptographyDeprecationWarning: Blowfish has been deprecated
  "class": algorithms.Blowfish,
/home/dbdimitrov/anaconda3/lib/python3.8/site-packages/numpydoc/docscrape.py:449: UserWarning: potentially wrong underline length... 
Extended Summary 
---------- in 
Add a Points layer to the layer list. 
...
  warn(msg)
/home/dbdimitrov/anaconda3/lib/python3.8/site-packages/numpydoc/docscrape.py:449: UserWarning: potentially wrong underline length... 
Extended Summary 
---------- in 
Add a Labels layer to the layer list. 
...
  warn(msg)
/home/dbdimitrov/anaconda3/lib/python3.8/site-packages/numpydoc/docscrape.py:449: UserWarning: potentially wrong underline length... 
Extended Summary 
---------- in 
Add a Shapes layer to the layer list. 
...
  warn(msg)
/home/dbdimitrov/anaconda3/lib/python3.8/site-packages/numpydoc/docscrape.py:449: UserWarning: potentially wrong underline length... 
Extended Summary 
-------

In [2]:
import liana as li

In [3]:
from matplotlib.pyplot import hist

In [4]:
from liana.method._global_lr_pipe import _global_lr_pipe
from liana.method.sp._spatialdm import _get_ordered_matrix, _standardize_matrix

In [ ]:
counts = pd.read_csv("data/counts_mat.csv")
weights = pd.read_csv("data/weight_mat.csv")

In [ ]:
var = pd.DataFrame(counts[['Unnamed: 0']]).set_index('Unnamed: 0')
var.index.name = None

In [ ]:
adata = sc.AnnData(X=csr_matrix(counts.drop(counts.columns[0], axis=1), dtype=np.float32).T, var=var)

In [ ]:
adata.obsm['proximity'] = csr_matrix(weights)

In [5]:
# scHOT data test
adata = sc.read_h5ad("test_spatial.h5ad")
resource = pd.DataFrame({"ligand":["Dnm1l", "Arrb1", "Igf2", "Dnm1l"], "receptor":["Gucy1b3", "Mtor", "Tuba1a", "Fam63b"]})
dist = adata.obsm['proximity']

In [ ]:
# full visium slide
# load the pre-processed dataset
img = sq.datasets.visium_hne_image()
adata = sq.datasets.visium_hne_adata()

li.mt.get_spatial_proximity(adata=adata, parameter=200, bypass_diagonal=False, cutoff=0.1)
dist = adata.obsm['proximity']

my_p = li.pl.proximity_plot(adata, idx=100)
resource = li.resource.select_resource("mouseconsensus")

In [6]:
temp, lr_res, ligand_pos, receptor_pos = _global_lr_pipe(adata=adata,
                                                         resource=resource,
                                                         expr_prop=0.05,
                                                         use_raw=False,
                                                         verbose=True,
                                                         layer=None,
                                                         _key_cols=['ligand_complex', 'receptor_complex'],
                                                         _complex_cols=['ligand_means', 'receptor_means'],
                                                         _obms_keys=['proximity'],
                                                         resource_name=None
                                                         )

Using `.X`!
Converting mat to CSR format


Correlations

In [11]:
# ligand-receptor mats
ligand_mat = _get_ordered_matrix(temp.X, ligand_pos, lr_res.ligand)
receptor_mat = _get_ordered_matrix(temp.X, receptor_pos, lr_res.receptor)

In [12]:
ligand_mat, receptor_mat = ligand_mat.T, receptor_mat.T

masked

In [13]:
import scipy.stats as stats

In [ ]:
def masked_wcor(x, y, weight, method='spearman_nzw'):
    spot_n = x.shape[1]
    
    # reshape x and y to be the same shape as weight
    x = np.reshape(np.repeat(x, spot_n), newshape=(spot_n, spot_n)).T
    y = np.reshape(np.repeat(y, spot_n), newshape=(spot_n, spot_n)).T
    
    # mask x and y with the same mask as weight
    x_masked = np.ma.array(x, mask = weight.mask, fill_value=np.nan)
    y_masked = np.ma.array(y, mask = weight.mask, fill_value=np.nan)
    
    if method == 'spearman_nzw':
        x_masked = stats.mstats.rankdata(x_masked, axis=1)
        y_masked = stats.mstats.rankdata(y_masked, axis=1)
        
    
    # calculate weighted pearson correlation
    wsum = np.ma.sum(weight, axis=1)
    
    n1 = wsum * np.ma.sum(weight * x_masked * y_masked, axis=1)
    n2 = np.ma.sum(weight * x_masked, axis=1) * np.ma.sum(weight * y_masked, axis=1)
    numerator = n1 - n2
    
    denominator_x = wsum * np.ma.sum(weight * (x_masked**2), axis=1) - np.ma.sum(weight * x_masked, axis=1)**2
    denominator_y = wsum * np.ma.sum(weight * (y_masked**2), axis=1) - np.ma.sum(weight * y_masked, axis=1)**2
    wcor = numerator / np.ma.sqrt(denominator_x * denominator_y)
    
    return wcor.data

In [ ]:
def calculate_masked_correlations(x_mat, y_mat, dist, method='spearman_nzw'):
    weight = dist.A
    msk = np.logical_not(weight>0).astype(np.int16)
    weight = np.ma.masked_array(weight, mask=msk)
    # calculate for each x and y combination
    local_correlations = []
    for i in range(x_mat.shape[0]):
        local_correlations.append(masked_wcor(x_mat[i, :], y_mat[i, :], weight=weight))
    local_correlations = np.array(local_correlations)
    
    return local_correlations


In [ ]:
%%time

dist = adata.obsm['proximity']

masked_sp = calculate_masked_correlations(ligand_mat.A[0:10,:], receptor_mat.A[0:10,:], dist=dist, method='spearman_nzw')

In [ ]:
masked_sp

Fully-vectorized

In [ ]:
import squidpy as sq
import scanpy as sc
import pandas as pd
import numpy as np
from scipy.sparse import csr_matrix, isspmatrix_csr

In [ ]:
import liana as li

In [ ]:
from matplotlib.pyplot import hist

In [ ]:
from liana.method._global_lr_pipe import _global_lr_pipe
from liana.method.sp._spatialdm import _get_ordered_matrix, _standardize_matrix

In [ ]:
from scipy.stats import rankdata

In [ ]:
# ligand-receptor mats
ligand_mat = _get_ordered_matrix(temp.X, ligand_pos, lr_res.ligand)
receptor_mat = _get_ordered_matrix(temp.X, receptor_pos, lr_res.receptor)

In [ ]:
def calculate_local_correlations(x_mat, y_mat, dist, method="pearson"):
    if method not in ["pearson", "spearman"]:
        raise ValueError("method must be one of 'pearson', 'spearman'")
    
    weight = dist.A.T
    weight_sums = np.sum(weight, axis = 0).flatten()
        
    if method=="spearman":
        x_mat = rankdata(x_mat, axis=1)
        y_mat = rankdata(y_mat, axis=1)
    
    # standard pearson
    n1 = (((x_mat * y_mat).dot(weight)) * weight_sums)
    n2 = (x_mat.dot(weight)) * (y_mat.dot(weight))
    numerator = n1 - n2
    
    denominator_x = (weight_sums * (x_mat ** 2).dot(weight)) - (x_mat.dot(weight))**2
    denominator_y = (weight_sums * (y_mat ** 2).dot(weight)) - (y_mat.dot(weight))**2
    denominator = np.sqrt(denominator_x * denominator_y)
    denominator[denominator == 0] = np.finfo(float).eps # add noise to avoid division by zero
    
    local_corrs = (numerator / denominator)
    
    return local_corrs

In [ ]:
dist = adata.obsm['proximity']

In [ ]:
local_pc = calculate_local_correlations(x_mat = ligand_mat.A, y_mat=receptor_mat.A, dist=dist, method="pearson")
local_sp = calculate_local_correlations(x_mat = ligand_mat.A, y_mat=receptor_mat.A, dist=dist, method="spearman")

In [ ]:
local_pc

In [ ]:
from scipy.stats import spearmanr,  pearsonr

In [ ]:
hist(local_sp[0,:])

local p-values

In [ ]:
from numpy import random
from tqdm import tqdm

In [ ]:
rng = random.default_rng(0)
n_perm = 100
positive_only = True # remove this option?

In [ ]:
dist = adata.obsm['proximity']

In [ ]:
local_pc.shape

In [ ]:
def _get_local_permutation_pvals(x_mat, y_mat, local_truth, local_fun, dist, n_perm, positive_only=True, **kwargs):
    xy_n = local_truth.shape[0]
    spot_n = local_truth.shape[1]
    
    # permutation cubes to be populated
    local_pvals = np.zeros((xy_n, spot_n))
    
    # shuffle the matrix
    for i in tqdm(range(n_perm)):
        _idx = rng.permutation(spot_n)
        perm_score = local_fun(x_mat = x_mat[_idx, :], y_mat=y_mat, dist=dist, **kwargs) ## TODO switch to shuffle rows, not columns
        if positive_only:
            local_pvals += np.array(perm_score >= local_truth, dtype=int)
        else:
            local_pvals += (np.array(np.abs(perm_score) >= np.abs(local_truth), dtype=int))

    local_pvals = local_pvals / n_perm
    
    return local_pvals
    

In [ ]:
local_pvals = _get_local_permutation_pvals(ligand_mat.A, 
                                           receptor_mat.A, 
                                           local_pc,
                                           calculate_local_correlations,
                                           dist, 
                                           n_perm, 
                                           positive_only=False)

In [ ]:
from  scipy.sparse import csr_matrix

In [ ]:
local_pvals.shape

In [ ]:
local_pvals

In [ ]:
local_pc.shape

In [ ]:
local_pvals.shape

In [ ]:
local_masked_pvals = _get_local_permutation_pvals(x_mat = ligand_mat.A,
                                                  y_mat = receptor_mat.A,
                                                  local_truth = masked_sp,
                                                  local_fun=calculate_masked_correlations,
                                                  dist=dist,
                                                  n_perm=n_perm,
                                                  positive_only=False)

In [ ]:
local_masked_pvals

In [ ]:
local_masked_pvals.shape

In [ ]:
from scipy.stats import spearmanr,  pearsonr

In [ ]:
pearsonr(local_masked_pvals[0,:], local_pvals[0,:])